In [16]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

In [ ]:

df_DST = pd.read_csv("../data/DST.csv")
df_K = pd.read_csv("../data/K.csv")
df_QB = pd.read_csv("../data/QB.csv")
df_RB = pd.read_csv("../data/RB.csv")
df_TE = pd.read_csv("../data/TE.csv")
df_WR = pd.read_csv("../data/WR.csv")

df_DST['Position'] = 'DST'
df_K['Position'] = 'K'
df_QB['Position'] = 'QB'
df_RB['Position'] = 'RB'
df_TE['Position'] = 'TE'
df_WR['Position'] = 'WR'

df = pd.concat([df_DST, df_K, df_QB, df_RB, df_TE, df_WR], ignore_index=True)

print(f"Total de filas: {len(df)}")
print(f"\nDistribución por posición:")
print(df['Position'].value_counts())
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nColumnas:")
print(df.columns.tolist())
print(f"\nInfo del DataFrame:")
print(df.info())

Total de filas: 938

Distribución por posición:
Position
WR     325
RB     208
TE     192
QB     121
K       58
DST     34
Name: count, dtype: int64

Primeras filas:
   Rank                      Player  SACK  INT   FR   FF  DEF TD  SFTY  \
0   1.0      Seattle Seahawks (SEA)  12.0  7.0  0.0  0.0     0.0   0.0   
1   2.0  Jacksonville Jaguars (JAC)   7.0  9.0  4.0  5.0     0.0   0.0   
2   3.0     Minnesota Vikings (MIN)  11.0  2.0  5.0  8.0     2.0   0.0   
3   4.0   Philadelphia Eagles (PHI)   5.0  3.0  2.0  4.0     0.0   0.0   
4   5.0         Detroit Lions (DET)  14.0  3.0  3.0  4.0     0.0   0.0   

   SPC TD    G  ...  TD  SACKS ATT.1 YDS.1  TD.1  FL  20+  TGT  REC  Y/R  
0     2.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
1     1.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
2     0.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
3     2.0  4.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
4     1.0  4.0

In [ ]:
# ---------- 1) Definir target ----------
TARGET = "FPTS"

# ---------- 2) Quitar columnas que NO deben ser features ----------
# - Identificadores y texto
id_like = ["Player", "Team"]  # agrega otras si las tienes (e.g., 'PlayerId')
# - Fugas de información (derivadas del target o rankings)
leak_like_patterns = [
    r"^FPTS\/G$",      # puntos por juego (deriva del target)
    r"rank",           # cualquier 'rank' o variantes
    r"tier",           # tiers si existieran
]
# Compilar regex para filtrar
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df.columns if leak_regex.search(str(c))])

# ---------- 3) Seleccionar columnas numéricas y categóricas ----------
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]

# Asegurar que Position esté como categórica
cat_cols = ["Position"]

# ---------- 4) Imputación + OneHot para 'Position' ----------
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

# ---------- 5) Ajustar transformador y generar X, y ----------
X = preprocessor.fit_transform(df)
y = df[TARGET].values

# ---------- 6) Obtener nombres de features transformadas ----------
ohe_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
feature_names = np.r_[num_cols, ohe_feature_names]

print(f"Total features numéricas: {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(ohe_feature_names)}")
print(f"Total de features finales: {len(feature_names)}")

# Vista rápida de las primeras 25 columnas transformadas
print("\nEjemplo de nombres de features resultantes:")
print(feature_names[:25])


Total features numéricas: 32
Total features categóricas (one-hot): 6
Total de features finales: 38

Ejemplo de nombres de features resultantes:
['SACK' 'INT' 'FR' 'FF' 'DEF TD' 'SFTY' 'SPC TD' 'G' 'FG' 'FGA' 'PCT' 'LG'
 '1-19' '20-29' '30-39' '40-49' '50+' 'XPT' 'XPA' 'CMP' 'ATT' 'Y/A' 'TD'
 'SACKS' 'ATT.1']


In [ ]:
# ------------------ 0) Configuración ------------------
TARGET = "FPTS"

# ------------------ 1) Asegurar tipos numéricos y target limpio ------------------
df = df.copy()

# Forzar columnas numéricas a ser realmente numéricas cuando aplique (sin romper strings válidos)
# Aquí convertimos solo el TARGET explícitamente; el resto lo manejará el ColumnTransformer con imputación
df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")

# Quitar filas con FPTS NaN/inf
mask = np.isfinite(df[TARGET])
df = df.loc[mask].reset_index(drop=True)

print(f"Filas después de limpiar {TARGET}: {len(df)}")

Filas después de limpiar FPTS: 926


In [4]:

# ------------------ 2) Definir columnas a eliminar (no-features) ------------------
# Identificadores y texto que no deben entrar como features
id_like = ["Player", "Team"]  # si faltan, se manejan luego para imprimir
# Fugas de información: cualquier cosa derivada del target o rankings
leak_like_patterns = [
    r"^FPTS\/G$",   # puntos por juego (deriva del target)
    r"\brank\b",    # rank, Rank, RANK
    r"\btier\b",    # tier, Tier, TIER
]
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df.columns if leak_regex.search(str(c))])

In [5]:
# ------------------ 3) Columnas numéricas y categóricas ------------------
# Asegurar que Position exista
if "Position" not in df.columns:
    raise ValueError("No se encontró la columna 'Position' en el DataFrame.")

num_cols = [c for c in df.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]
cat_cols = ["Position"]

print(f"Total features numéricas (detectadas): {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(cat_cols)} → {cat_cols}")

Total features numéricas (detectadas): 32
Total features categóricas (one-hot): 1 → ['Position']


In [6]:
# ------------------ 4) Preprocesamiento ------------------
# Imputación numérica con 0 (robusto para ausencias por posición)
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)

# Para categórica: imputar más frecuente y luego OneHot (evita NaNs en OHE)
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

In [7]:
# ------------------ 5) Split train/test con el df crudo (el preprocessor hará la transformación) ------------------
X_train_df, X_test_df, y_train, y_test = train_test_split(
    df.drop(columns=[TARGET]), df[TARGET], test_size=0.2, random_state=42
)

# ------------------ 6) Modelo XGBoost ------------------
model_xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model_xgb)
])

# Entrenar
xgb_pipeline.fit(X_train_df, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [12]:
# ------------------ 7) Evaluación ------------------
y_pred_test = xgb_pipeline.predict(X_test_df)
rmse = mean_squared_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
print(f"RMSE (test): {rmse:.2f}")
print(f"R² (test):   {r2:.3f}")

RMSE (test): 18.41
R² (test):   0.942


In [13]:
# ------------------ 8) Importancias de características ------------------
# Recuperar nombres de columnas transformadas
num_cols_fitted = preprocessor.transformers_[0][2]
cat_cols_fitted = preprocessor.transformers_[1][2]
ohe_names = xgb_pipeline.named_steps["preprocessor"] \
                        .named_transformers_["cat"] \
                        .named_steps["ohe"] \
                        .get_feature_names_out(cat_cols_fitted)
feature_names_fitted = np.r_[num_cols_fitted, ohe_names]

importances = xgb_pipeline.named_steps["model"].feature_importances_
feat_imp = pd.DataFrame({"Feature": feature_names_fitted, "Importance": importances}) \
            .sort_values("Importance", ascending=False)

print("\n--- Top 15 Features más importantes ---")
print(feat_imp.head(15).reset_index(drop=True))



--- Top 15 Features más importantes ---
         Feature  Importance
0   Position_DST    0.311815
1             TD    0.261416
2            ATT    0.074204
3            CMP    0.062524
4             LG    0.042492
5              G    0.042059
6            REC    0.036152
7    Position_RB    0.022509
8            FGA    0.021670
9           SACK    0.018711
10            FG    0.018016
11         ATT.1    0.016477
12           INT    0.012113
13         YDS.1    0.010698
14          TD.1    0.007944


In [ ]:
# ------------------ 9) Predicciones completas y Top 10 Jugadores ------------------
df_pred = df.copy()

# Crear columna Team si viene con alias común
if "Team" not in df_pred.columns:
    for alt in ["Tm", "TEAM", "TeamAbbrev", "Franchise", "Club"]:
        if alt in df_pred.columns:
            df_pred["Team"] = df_pred[alt]
            break

# Crear columna Player si viene con alias
if "Player" not in df_pred.columns:
    for alt in ["Name", "PLAYER", "PlayerName"]:
        if alt in df_pred.columns:
            df_pred["Player"] = df_pred[alt]
            break

# Predicción sobre todo el dataset limpio
df_pred["Pred_FPTS"] = xgb_pipeline.predict(df.drop(columns=[TARGET]))

# Top 10
top_pred = df_pred.sort_values("Pred_FPTS", ascending=False).head(20)

# Seleccionar columnas disponibles para imprimir sin romper
display_cols = ["Player", "Position", "Team", "Pred_FPTS", "FPTS"]
available_cols = [c for c in display_cols if c in top_pred.columns]

print("\n--- Top 10 Jugadores Predichos por FPTS ---")
print(top_pred[available_cols].reset_index(drop=True))


--- Top 10 Jugadores Predichos por FPTS ---
                    Player Position  Pred_FPTS  FPTS
0         Josh Allen (BUF)       QB  98.937622  99.5
1      Lamar Jackson (BAL)       QB  93.878510  94.4
2  Patrick Mahomes II (KC)       QB  89.397346  89.6
3          Drake Maye (NE)       QB  86.014954  85.5
4        Jalen Hurts (PHI)       QB  84.346504  84.2
5     Caleb Williams (CHI)       QB  84.308907  84.1
6       Daniel Jones (IND)       QB  80.052292  80.5
7      Baker Mayfield (TB)       QB  79.908073  80.1
8     James Cook III (BUF)       RB  78.157944  79.0
9         Jordan Love (GB)       QB  75.189911  75.2
